In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import math
import quandl
import collections, functools, operator
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from operator import itemgetter
pd.set_option('display.max_columns', 500)
pd.set_option('mode.chained_assignment', None)
%matplotlib inline

In [2]:
def data_cleaning(data):
    '''
    @author: Wuding Li
    @The function cleans the data
    '''
    
    data = data[data['date'] != 19251231]
    #counts = data['date'].value_counts().to_dict()
    
    #adjust date format
    data['date'] = data['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    
    
    #remove PRC and SHROUT missing. According to the describtion, PRC missing meanings totally missing without a clue
    data['PRC'] = pd.to_numeric(data.PRC, errors='coerce')
    data = data.dropna(subset = ['PRC'])
    data['PRC'] = data['PRC'].abs()
    
    data['SHROUT'] = pd.to_numeric(data.SHROUT, errors='coerce')
    data = data.dropna(subset = ['SHROUT'])
    
    #get the medium
    NYSE = pd.read_excel('NYSE medium.xlsx')
    NYSE['date'] = pd.to_datetime(NYSE['date'])
    data['YearMonth'] = data['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE['YearMonth'] = NYSE['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE = NYSE.drop(columns=['date'])    
    data = pd.merge(data,NYSE,how = 'left',on = 'YearMonth')
    data = data[data['YearMonth'] != '192512']
    #counts = data['date'].value_counts().to_dict()
    

    data['mkt_cap'] = data['PRC']*data['SHROUT']
    data = data[data['mkt_cap']*1000>data['market cap']] #SHROUT is approximate to 10**3

    
    #Remove ADR CLosed end fund and foreign fund
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.ffill())
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.bfill())
    data = data[data['SHRCD'].isin([11.0,10.0])]
    
    
    #check more than 5
    mask = (data['PRC'] > 5)
    data = data.loc[mask]

    #clean the RET and RETX column. 'C' stands for the first trading month. Then we change it to 0
    data['RET'] = data['RET'].apply(pd.to_numeric, errors='coerce').fillna(0)
    data['RETX'] =data['RETX'].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    data = data.drop(columns=['YearMonth'])


    return data

In [3]:
def rolling_time_window(Start_Month, Start_Year, Time_Period_Training, Time_Period_Testing, Gap, data):
    '''
    @author: Wuding Li
    @The function returns two cleaned datasets. One is the sampling period data and the other is testing period data
    @variables: Start_Month: the starting month of our rolling time period
                Start_Year: the starting year of our rolling time period
                Time_Period_Training: Span of sampling Period
                Time_Period_Testing: Span of testing Period
                Gap: Gap between sampling and testing
                Data: The input data dataset
    @Also this function apply the filter no closed end fund and stock price more than 5 dollars
    
    @Updated: 10/7 fixed the end_date_test "frequency = 0" problem
    @Updated: 10/16 change "DATE" to "date" to match the updated format
    @Updated: 11/1 used the French's medium to substitute our formal medium which directly get from dataset
    '''
    
    #get the start and end date of data
    start_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=1, freq='D')[0].date())
    end_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=2, freq=str(Time_Period_Training-1)+'M')[-1].date())
    mask_train = (data['date'] >= start_date_train) & (data['date'] <= end_date_train)
    
    
    start_month_testing = Start_Month+Time_Period_Training+Gap
    if start_month_testing % 12 == 0:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)-1
        start_month_testing = 12
    else:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)
        start_month_testing = start_month_testing % 12    
    start_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq='D')[0].date())
    end_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq=str(Time_Period_Testing)+'M')[-1].date())
    mask_test = (data['date'] >= start_date_test) & (data['date'] <= end_date_test)
    
    #applying filter functions below on the master dataset
    mask_master = (data['date'] >= start_date_train) & (data['date'] <= end_date_test)
    data_master = data.loc[mask_master]
    

    full = []
    for i in data_master['PERMNO'].values:
        if (data_master[data_master['PERMNO'] == i]).shape[0] == (Time_Period_Training+ Time_Period_Testing+ Gap):
            full.append(i)
    data_master = data_master[data_master['PERMNO'].isin(full)]

    
    return data_master.loc[mask_train],data_master.loc[mask_test]


In [4]:
def format_FF():
    '''
    @author: Zhikang Wang
    @The function read and clean up the fama_french dataset pulled from Dr. French's website
    @Variable: input is fama_french dataset pulled from Dr. French's website
    '''
    
    ff_data = pd.read_csv("F-F_5.CSV")
    # Getting and renaming the columns we need
    ff_ret = ff_data[['YearMonth','Mkt-RF','SMB','HML','RMW','CMA','RF']]
    ff_ret = ff_ret.rename(columns = {"Mkt-RF": "mkt_ret"})
    # converting the factor exposures to percentage
    ff_ret["mkt_ret"] = ff_ret["mkt_ret"]/100
    ff_ret["SMB"] = ff_ret["SMB"]/100
    ff_ret["HML"] = ff_ret["HML"]/100
    ff_ret["RF"] = ff_ret["RF"]/100
    ff_ret["RMW"] = ff_ret["RMW"]/100
    ff_ret["CMA"] = ff_ret["CMA"]/100
    ff_ret["YearMonth"] = ff_ret["YearMonth"].astype(str) #converting YearMonth to string
    
    return ff_ret

In [5]:
def get_weight(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)  
    @update: 11/17/2019 Wuding Li updated the weight function by Lehmann(1990)
    '''
    #count = len(set(data['PERMNO'].values))  
    data['weight_temp'] = data.groupby(['date'])['RET'].apply(lambda x: (x-x.mean()))
    data['weight_temp1']= data['weight_temp'].apply(lambda x: 0 if x<0 else x)
    data['weight_temp1'] = data['weight_temp1'].groupby(data['date']).transform('sum')
    data['weight'] = data['weight_temp']/data['weight_temp1']
    data = data.drop(columns=['weight_temp','weight_temp1'])
    data['weight'] = data.groupby(['PERMNO'])['weight'].shift(1)
    data = data.dropna(subset = ['weight'])
    return data

In [6]:
def get_weight_Residual(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the residual's weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    
    count = len(set(data['PERMNO'].values))
    def resd_avg(df):
        result = (df['residual'] - df['residual_mean'])
        return result
    
    temp_output = pd.DataFrame()
    YearMonth_list = list(set(data['YearMonth'].values.tolist()))
    for i in YearMonth_list:
        temp = data[data['YearMonth'] == i]
        temp['residual_weight'] = (temp['residual'] - temp['residual_mean'])
        temp['residual_weight_winner'] = temp['residual_weight'].apply(lambda x: 0 if x<0 else x)
        temp['residual_real_weight'] = temp['residual_weight']/(temp['residual_weight_winner'].sum())
        temp = temp.drop(columns=['residual_weight','residual_weight_winner'])
        temp = temp.rename(columns = {"residual_real_weight": "residual_weight"})
        temp_output = temp_output.append(temp)
        
    temp_output = temp_output.sort_values(by=['YearMonth'])
    temp_output['residual_weight'] = temp_output.groupby(['PERMNO'])['residual_weight'].shift(1)
    temp_output = temp_output.dropna(subset = ['residual_weight'])
    return temp_output

In [7]:
def get_equal_weight(data):
    '''
    @author: Zhikang Wang
    @The function add a column to the data set that assigned equal weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    data['1'] = [1]*len(data)
    data['equal_weight']=data.groupby(['date'])['1'].apply(lambda x: x*(1/count)).shift(1)
    data = data.dropna()
    return data

In [8]:
def regression_model(combined_temp):
    '''
    @author: Robin Lam, Yulin Chen, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        combined_temp: dataframe which contains returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        combined_temp: same dataframe as the input, but with an extra column (residual)
    
    @Update: 10/16/2019: Wuding Li Change the output date to match the paper 
    '''
    x = combined_temp[['mkt_ret','SMB','HML','RMW','CMA']]
    y = combined_temp['ticker_excess_ret']
    
    # Linear regression model
    x = sm.add_constant(x)
    regressor = sm.OLS(y, x).fit()
    # retrieving the slope
    coef = regressor.params
    # retrieving the t-statistics
    t_test = regressor.pvalues
    # retrieving the r-squared
    r_squared = regressor.rsquared_adj
    
    
    # calculating the residuals for each regression model
    combined_temp['residual'] = y - x['mkt_ret']*coef[1] - x['SMB']*coef[2] - x['HML']*coef[3] - x['RMW']*coef[4] - x['CMA']*coef[5] - coef[0]
    standard_dev = combined_temp['residual'].std() #get the sd of the residual
    avg_ret = combined_temp['residual'].mean()
    combined_temp['residual_sd'] = (combined_temp['residual']-avg_ret)/standard_dev
    combined_temp['residual_mean'] = combined_temp['residual_sd'].mean()
    
    combined_temp['mkt_factor'] = x['mkt_ret']*coef[1]    #used in conditiona_factor_model
    combined_temp['SMB_factor'] = x['SMB']*coef[2]    #used in conditiona_factor_model
    combined_temp['HML_factor'] = x['HML']*coef[3]    #used in conditiona_factor_model
    combined_temp['RMW_factor'] = x['RMW']*coef[4]
    combined_temp['CMA_factor'] = x['CMA']*coef[5]
    
    # adding coefficients to columns
    combined_temp['alpha'] = coef[0]
    combined_temp['mkt_beta'] = coef[1]
    combined_temp['SMB_beta'] = coef[2]
    combined_temp['HML_beta'] = coef[3]
    combined_temp['RMW_beta'] = coef[4]
    combined_temp['CMA_beta'] = coef[5]
    combined_temp['r_squared'] = r_squared
    
    # adding t-statistics to columns
    combined_temp['t_alpha'] = t_test[0]
    combined_temp['t_mkt_beta'] = t_test[1]
    combined_temp['t_SMB_beta'] = t_test[2]
    combined_temp['t_HML_beta'] = t_test[3]
    combined_temp['t_RMW_beta'] = t_test[4]
    combined_temp['t_CMA_beta'] = t_test[5]
    
    combined_temp['port_mkt_beta'] = combined_temp['mkt_beta']*combined_temp['weight']
    combined_temp['port_SMB_beta'] = combined_temp['SMB_beta']*combined_temp['weight']
    combined_temp['port_HML_beta'] = combined_temp['HML_beta']*combined_temp['weight']
    combined_temp['port_RMW_beta'] = combined_temp['RMW_beta']*combined_temp['weight']
    combined_temp['port_CMA_beta'] = combined_temp['CMA_beta']*combined_temp['weight']
    
      
#     keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
#     output = combined_temp[combined_temp['YearMonth'].isin(keep)]
    return combined_temp, standard_dev

In [9]:
def all_regression(df):
    '''
    @author: Robin Lam, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        df: dataframe which contains ALL of the stocks' returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        output: same dataframe as the input, but with an extra column (residual)
        dict_coef: dictionary of coefficients from all the regression for ALL stocks. should have 3 betas.
        
    @Update: 10/16/2019: Wuding Li Change the output format of the function and the structure of the function
    '''
    output = pd.DataFrame()
    #output_temp = pd.DataFrame()
    permno_list = list(set(df['PERMNO'].values.tolist()))
    sd_dic = {}
    for i in permno_list:
        temp_train = df[df['PERMNO'] == i]
        combined_temp, sd = regression_model(temp_train)    #there are 36 months in temp_train (individual ticker)
        #output_temp = output_temp.append(combined_temp)
        keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
        output_keep = combined_temp[combined_temp['YearMonth'].isin(keep)]
        sd_dic[i] = sd
        output = output.append(output_keep)
    output = get_weight_Residual(output)   #this is also returned to merge with the test dataset for later!
    
    return sd_dic, output

In [10]:
msf = pd.read_csv('msf.csv')
msf_before = msf

In [11]:
msf = data_cleaning(msf)

In [19]:
'''
Author: Zhikang Wang / Yulin Chen
MAIN FUNCTION.
Loop over the period from 1986 to generate rolling regression
'''

test_port = pd.DataFrame() #container for test df
output_conditional_output = pd.DataFrame()    #container for Grundy Table output
total_train_output = pd.DataFrame()

dates = []
ff = []
start = 1963      #1963

for i in range(457,642): #（7，232） （232，457） （457，642）
    if (i%12) != 0: 
        month = i%12
        year = i // 12 + start
    if (i%12) == 0: 
        month = 12
        year = i // 12 + start -1

    ff_ret = format_FF()
    train,test = rolling_time_window(month, year, 37, 1, 0, msf)
    #train = get_equal_weight(train)
    train = get_weight(train)
    
    train = train.rename(columns = {"RET": "ticker_ret", "date": "YearMonth"})
    train = train[['YearMonth','PERMNO','ticker_ret','weight']]
    train["PERMNO"] = train["PERMNO"].astype(str)
    test["PERMNO"] = test["PERMNO"].astype(str)
    train["YearMonth"] = train["YearMonth"].astype(str)
    train['YearMonth'] = train['YearMonth'].apply(lambda x: x[:4] + x[5:7])

    # Merging the two dataset into one combined return dataset, which will be used as regression input
    combined_ret = pd.merge(ff_ret, train, how='right', on=['YearMonth'])

    # calculates the excess return of tickers by subtracting the returns by risk free rate
    combined_ret['ticker_excess_ret'] = combined_ret['ticker_ret'] - combined_ret['RF']
    #append dates
    month_list=["01","02","03","04","05","06","07","08","09","10","11","12"]
    month=month_list[month-1]
#     year = year + 3
    str_date = str(year)+str(month)
    dates.append(str_date)
    
    sd_dic, output = all_regression(combined_ret)
    
    total_train_output = total_train_output.append(output)
    test_port = test_port.append(test)
    print(i)

457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641


In [20]:
export_csv1 = total_train_output.to_csv ('5_factor_train_output3.csv', index = None, header=True)
export_csv2 = test_port.to_csv ('5_factor_test_output3.csv', index = None, header=True)

In [21]:
total_train_output

,YearMonth,mkt_ret,SMB,HML,RMW,CMA,RF,PERMNO,ticker_ret,weight,ticker_excess_ret,residual,residual_sd,residual_mean,mkt_factor,SMB_factor,HML_factor,RMW_factor,CMA_factor,alpha,mkt_beta,SMB_beta,HML_beta,RMW_beta,CMA_beta,r_squared,t_alpha,t_mkt_beta,t_SMB_beta,t_HML_beta,t_RMW_beta,t_CMA_beta,port_mkt_beta,port_SMB_beta,port_HML_beta,port_RMW_beta,port_CMA_beta,residual_weight
23701,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,76240,0.090251,0.005385,0.089551,0.068598,0.686108,1.541976e-17,0.029334,0.011919,-0.027553,-0.019336,0.021917,0.004672,1.364390,0.469238,-1.398616,0.547763,0.644631,0.280486,0.835796,3.258667e-02,0.452969,1.025820e-01,0.547835,0.468712,0.007348,0.002527,-7.531919e-03,0.002950,3.471512e-03,0.007201
23470,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,40061,-0.015506,0.006743,-0.016206,-0.053800,-0.682836,-1.233581e-17,0.007391,0.008911,-0.002565,0.036615,-0.028516,0.015758,0.343769,0.350814,-0.130190,-1.037249,-0.838703,0.389699,0.377935,4.792385e-01,0.476267,8.436809e-01,0.154490,0.235104,0.002318,0.002366,-8.778960e-04,-0.006994,-5.655549e-03,0.009647
23545,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,52898,-0.013575,-0.001361,-0.014275,-0.013560,-0.265267,-1.696174e-17,0.003835,0.017365,-0.001583,-0.018940,-0.008809,0.007417,0.178392,0.683646,-0.080351,0.536544,-0.259094,0.017388,0.521104,5.708834e-01,0.038331,8.512058e-01,0.253371,0.568306,-0.000243,-0.000930,1.093547e-04,-0.000730,3.526179e-04,0.001916
23250,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,10401,-0.041379,-0.000368,-0.042079,-0.073919,-0.888962,3.083953e-18,0.008143,-0.026887,0.037292,0.042483,-0.036176,0.006985,0.378753,-1.058557,1.892992,-1.203492,-1.064008,0.248315,0.709654,4.602979e-01,0.047913,1.023612e-02,0.118771,0.155642,-0.000139,0.000389,-6.958785e-04,0.000442,3.911377e-04,-0.003687
23639,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,66181,-0.000564,-0.004732,-0.001264,-0.007771,-0.129089,-5.011423e-17,0.043923,0.002568,0.016018,-0.029516,-0.021115,-0.005370,2.042947,0.101085,0.813106,0.836160,-0.621033,0.597743,0.692578,4.626176e-06,0.787449,1.144365e-01,0.133762,0.249533,-0.009666,-0.000478,-3.847259e-03,-0.003956,2.938451e-03,-0.008951
23662,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,71271,0.047191,0.002470,0.046491,0.030039,0.837406,6.167906e-18,0.006761,0.004162,0.005828,-0.002516,0.004408,-0.002190,0.314455,0.163847,0.295829,0.071278,0.129657,0.047702,0.786538,1.603615e-01,0.465045,3.287900e-01,0.826997,0.683689,0.000777,0.000405,7.305721e-04,0.000176,3.201989e-04,0.004527
23538,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,52329,-0.065195,0.000232,-0.065895,-0.115298,-1.611808,-6.167906e-18,0.004938,-0.002399,-0.000780,0.007250,0.030292,0.010101,0.229694,-0.094465,-0.039569,-0.205384,0.890939,-0.023315,0.532250,6.018754e-01,0.832035,9.473536e-01,0.752259,0.166590,0.000053,-0.000022,-9.163689e-06,-0.000048,2.063302e-04,0.000995
23518,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,48960,-0.005816,-0.000079,-0.006516,-0.029656,-0.717994,3.083953e-17,0.020215,0.006041,0.001477,-0.015392,0.010329,0.000470,0.940212,0.237849,0.074981,0.436032,0.303780,0.348890,0.959711,7.802673e-04,0.358291,8.285039e-01,0.250688,0.409125,-0.000074,-0.000019,-5.911223e-06,-0.000034,-2.394901e-05,0.000017
23328,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,18092,0.067366,-0.001861,0.066666,0.078080,1.220059,6.167906e-18,0.006051,0.001672,0.004907,-0.021382,-0.020158,0.017497,0.281420,0.065818,0.249111,0.605728,-0.592886,-0.055264,0.230703,4.758206e-01,0.868784,6.427685e-01,0.302016,0.300068,-0.000524,-0.000122,-4.636036e-04,-0.001127,1.103378e-03,-0.001171
23574,200401,0.0215,0.0254,0.0197,-0.0353,0.0340,0.0007,58683,-0.073730,-0.009167,-0.074430,-0.077414,-1.062532,-3.700743e-17,0.036432,0.005280,-0.006585,-0.032805,0.013614,-0.012951,1.694508,0.207857,-0.334259,0.929326,0.400409,0.353462,0.432617,6.260785e-04,0.647268,5.848671e-01,0.167287,0.536340,-0.015534,-0.001905,3.064167e-03,-0.008519,-3.670565e-0